In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

In [ ]:
# LDDMM landmark dynamics
import src.params as params
params.manifold = 'landmarks'

In [ ]:
from src.manifold import *
from src.metric import *

from src.plotting import *
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
# Example with 2 landmarks:
k_sigma.set_value(np.diag((0.1,0.1)))

N.set_value(2)
x0 = np.array([[0,1],[1,1]])
q0 = x0.flatten()
print(q0)

vn0 = np.array([[1,1.],[-1,1]])#np.array([[1./2,1],[-1,1]])
print(vn0)
v0 = vn0 #GramSchmidt(vn0.flatten(),q0).reshape((2,2))

p0 = gMflatf(q0,v0.flatten()).astype(theano.config.floatX)
# initialize
#N.set_value(3)
#k_sigma.set_value(np.diag((.2,.2)))
#n_steps.set_value(100)

In [ ]:
# setup
x0 = np.vstack((np.linspace(-.5,.5,N.eval()),np.zeros(N.eval()))).T
q0 = x0.flatten()
v0 = np.vstack((np.zeros(N.eval()),np.ones(N.eval()))).T
p0 = gMflatf(q0,v0.flatten())
print("q = ", q0)
print("p = ", p0)

plt.plot(x0[:,0],x0[:,1], 'ro')
plt.quiver(x0[:,0],x0[:,1],v0[:,0],v0[:,1], scale = 2)
plt.axis((np.min(x0[:,0])-0.5,np.max(x0[:,0])+0.5,np.min(x0[:,1])-0.1,
          np.max(x0[:,1])+0.5))

In [ ]:
# Curvature
from src.curvature import *
print("Shape of curvature tensor = ", Rf(q0).shape)
#print("Sectional curvature = ", R_uif(q0,np.hstack((v0,v0)).reshape((4,2))))

In [ ]:
# Geodesic equation
from src.geodesic_equation import *
# test geodesic
plt.rcParams['figure.figsize'] = 7, 7
plt.axis('equal')
qsv = Expteqf(q0,p0)
#qsv = Expteqf(q0,v0.flatten())
plotx(qsv,p0,linewidth=1.5)
plt.show()
#plt.savefig("geoeqland.pdf")

In [ ]:
# Hamiltonian dynamics
from src.Hamiltonian import *
# test Hamiltionian and gradients
print(p0)
print(Hf(q0,p0))

# geodesic
qsv = Exptf(q0,p0).T
%matplotlib inline
plt.figure()
plt.axis('equal')
plotx(qsv,p0)
plt.show()
(ts,qpsv) = Hamf(q0,p0)
psv = qpsv[:,1,:]
print("Energy: ",np.array([Hf(q,p) for (q,p) in zip(qsv,psv)]))
#plt.savefig("landgeoH.pdf")

## Geodesic on frame bundle

In [ ]:
x1 = np.array([[0,1],[0.5,1]]).flatten()
ui0 = np.array([[1,0],[0,1],[-1,0],[0,1]])

#u0 = sp.linalg.orth(ui0[0:2])
#for i in range(1,int(d.eval()/2)):
#        foo = sp.linalg.orth(ui0[2*i:(2*i+2)])
#        u0 = np.append(u0,foo)
print(ui0)
q0 = np.hstack([x1,ui0.flatten()]).astype(theano.config.floatX)

xi0 = np.array([1,1,0,0])
xia = np.zeros_like(u0.flatten())
p0 = np.hstack([xi0,xia]).astype(theano.config.floatX)
print("q = ", q0)
print("p = ", p0)

x0 = np.array([q0,p0]).astype(theano.config.floatX)

In [ ]:
# Curvature
print("curvature = ", R_uif(x1,ui0))

In [ ]:
# Hamiltonian dynamics
from src.FM import *
# test Hamiltionian and gradients
print(p0)
print(Hfmf(q0,p0))

# geodesic
qsv = Exptfmf(q0,p0).T
%matplotlib inline
plt.figure()
plotFMx(qsv,N_vec=5)
plt.axis((-0.1,0.6,0.9,2.5))
plt.show()
(ts,qpsv) = Hamfmf(q0,p0)
psv = qpsv[:,1,:]
print("Energy: ",np.array([Hfmf(q,p) for (q,p) in zip(qsv,psv)]))
#plt.savefig("landgeoFM.pdf")

# Development and Horizontal Vector Fields

### Development

In [ ]:
n_steps.set_value(100)
x0 = np.array([[0,1],[0.5,1]])
v0 = np.array([[1,0,0,1],[-1,0,0,1]])
ui0 = v0 #GramSchmidt() #sp.linalg.orth(v0)

q0 = np.hstack([x0.flatten(),ui0.flatten()]).astype(theano.config.floatX)
print("q0 = ", q0)

# Curve in R^2:
t = np.linspace(0,10,n_steps.get_value()+1)
gamma0 = np.vstack([20*np.sin(t), t**2 + 2*t]).T
dgamma0 = np.diff(gamma0, axis = 0)
print("dgamma.shape =", dgamma0.shape)

In [ ]:
%matplotlib inline
plt.plot(gamma0[:,0],gamma0[:,1])
plt.plot(gamma0[0,0],gamma0[0,1], 'ro')
plt.plot(gamma0[-1,0],gamma0[-1,1],'go')
plt.axis((-21,21,-11,131))
#plt.savefig("landDev1.pdf")

In [ ]:
# Development dynamics
from src.Stochastic_Development import *
# test deterministic development
qsv = devf(q0,dgamma0)
plt.figure()
plotFMx(np.concatenate([q0.reshape((1,12)),qsv]),N_vec = 0)
plt.show()
#plt.savefig("landDev2.pdf")

### Stochastic development:

In [ ]:
n_steps.set_value(1000)
x0 = np.array([[0,1],[2,1]])
v0 = np.array([[1,0,0,1],[-1,0,0,1]])#np.array([[0,1,0,-1],[1,0,1,0]]).T

ui0 = v0 #sp.linalg.orth(v0)

q0 = np.hstack([x0.flatten(),ui0.flatten()]).astype(theano.config.floatX)
print("q0 = ", q0)

dWt0 = np.random.normal(0, np.sqrt(dt.eval()), (n_steps.get_value(),2))

drift0 = np.array([3,3])

In [ ]:
dZt = dWt0 + drift0*dt.eval()
Zt = np.concatenate([np.array([0,0]).reshape((1,2)),np.cumsum(dZt, axis = 0)])
plt.plot(Zt[:,0],Zt[:,1],'b-',linewidth = 1)
plt.plot(0,0, 'ro')
plt.plot(Zt[-1,0],Zt[-1,1], 'go')
#plt.savefig("landstoc1.pdf")

In [ ]:
# test stochastic development
qsv = stoc_devf(q0,dWt0,drift0)
plt.figure()

plotFMx(np.concatenate([q0.reshape((1,12)),qsv]),N_vec = 0)
plt.show()
#plt.savefig("landstoc2.pdf")